# Data Science Guild - Natural Language Processing (Part 1)


Natural language processing (NLP) is the algorithmic processing of natural language.  This tautology begs the question: what do we mean by natural language?  What do you think of when you hear the word "natural"?  Is a programming language natural?

Unnatural hello world in C++:

Natural hello world in Python:

`print "Natural Hello World!"`

While we can argue whether one programming language is more natural than another, it's tough to argue that they aren't inherently simpler than the equivalence of the following in English:

*Print "Hello World!" to the screen.*

or

*Kindly print the exclamation: "Hello World" to the terminal, followed by an exclamation point.*

By natural, we tend to think of the "natural" way one speaks.  When we think about natural language processing, we have a sense that it is somewhat more difficult to code against because there are a lot more possibilities to handle.  Programming languages are relatively simple with very explicit rules and a small number of keywords.  This difference between programming languages and natural language can be attributed to the differences in the grammars and vocabulary.  The grammar for english is much more "free" and less restrictive and the vocabulary is significantly larger.  Programming languages typically have around 50 keywords at most.  A working english vocabulary probably uses around 15,000 words.  Have you ever set out to process natural language in some way and first had this thought, "I know.  I'll build a bunch of regular expressions to try to match things." only to find that it sort of works up to a point, but then breaks down in many cases, probably exhausting you in the process from having to come up with more and more clever regexes?  Part of this is simply that regexes can be tricky, but part of this is a fundamental error in algorithmic assumptions.  We'll explain this error in a bit, but for know, just understand that there is a well-studied relationship between the complexity of the grammar that you are trying to parse and the type of parser that is sophisticated enough for the grammar.  This study of formal languages is a branch of computability and complexity theory that was largely instigated by Noam Chomsky and Marcel-Paul Schutzenberger's work in the late 1950s.  Their work created a hierarchy of grammars along with the associated automata that are required to regonize the grammar:
<img src="chomsky2.jpg">

The simpler the grammar, the simpler the algorithm that can recognize it.  So when we are trying to handle a phrase-structured language like English with regular expressions, which are suited for the much simpler class of regular grammars, we just don't have the firepower to recognize the constructs like we would like.

NLP is a discipline that also draws heavily on computability and information theory.  To gain an intuitive understanding of this relationship between information and language, read the following quote in which the consonants have been removed and decide who said it:

"ouoe a ee ea ao..."

Pretty tough, right?  Try the same thing with just the consonants:

"Frscr nd svn yrs g..."

One reason this is slightly easier is because the consonants together carry more information about the word than the vowels do.  This is why whole languages can exist, Hebrew for one, which written form contains no vowels, vowel pointings aside.

There are a lot of rabbit holes for us here.  Thankfully, it's not strictly necessary to have a complete understanding of automata theory and formal languages or information theory to get along in NLP.  It does, however, help considerably to have a general notion of the theoretical grounds you can stumble into as well as the kinds of software required to parse certain kinds of grammars, and generally where you are playing along the Chomsky hierarchy.

### Oversimplification as an NLP strategy

For a number of reasons, Occam's razor, avoiding hasty optimization, it makes a lot of sense to start with the simplest possible solution to extracting information from text by assuming that the language obeys certain rules, which it might not theoretically, but tends to do in practice.  This allows us to essentially code against a lower level on the hieararchy and still get pretty good results.  This tends to work because, for the most part, any kind of natural language you encounter is a subset of what can be produced by a grammar.  What can be produced, we typically call "syntax" while what makes sense we call "semantic".  Some syntactically correct sentences can be semantic nonsense.  The most famous example is probably Noam Chomsky's "Colorless green ideas sleep furiously."  But it's easy to imagine syntactically correct sentences that are semantic nonsense like "Invisible rocks smell the color five."

Let's see how this strategy can play out with an example I recently encountered while attempting to extract wedding/reception venue price information from free-text descriptions.  I started with a list of strings, each one representing the price of a single venue:

Looks daunting at first.  Descriptions are all over the place, sometimes they have rental fees, sometimes per-person fees, sometimes a range is indicated, not to mention how this stuff is ordered within the string.  But let's take a few examples and try to simplify them by cherry-picking keywords and ignoring everything else in order to get our minds around how to tackle this problem.  Take the first three, for example:

Now imagine you are like a three year old version of my son who loves trains and snakes and juice boxes.  As you encounter the events of your day, you can imagine that the wonders you encounter are simplified into a version that looks a lot like this:

_stuff_ _stuff_ _stuff_ _stuff_ **snake** _stuff_ _stuff_ _stuff_ **train** _stuff_ _stuff_ **juice box** _stuff_

Let's build a little recognizer that does that same kind of thing with our price strings.  First let's remove extraneous tokens and try to boil it down the simplest possible sentence that contains our information, preserving the order:

That worked pretty nicely, and in fact building a parser with this oversimplified view of reality seems tractable now.  Let's try a few more at random:

simplified becomes:

A little bit confusing here, but still much simpler than the original.  On the second string, it's hard to know what the range applies to.  But it's enough for our purposes to see that it is a per-person cost equivalent to a catering cost.  One might envision a recursive context-free grammar similar to the following to be able to handle these boiled down strings:

This grammar would be consumed by a tool that would generate a parser for you.  The parser would then produce a syntax tree from the grammar given an input string.  The pricing information could then be culled from the syntax tree in no worse than `O=lg(n)` time.  Our general process was:

* Clean the input
* Scan the input for keywords and price tokens
* Operate on the tokens and produce a parse tree based on our grammar

The second process is called `scanning` or `lexing` where the output is a sequence of `lexemes` or tokens to be processed, and the third process is called `parsing`.  Tools like lex/yacc or flex/bison can consume lists of regular expressions and EBNF grammars and produce parsers for you.

One very interesting connection here...  notice the production:

`<rental_price> -> <R> <price> | <price> <R>`

If you imagine reading a string and processing tokens from left to right, and if it could be assumed that the order of the price and rental keyword would always be the same, we wouldn't need both rules.  In fact, if strings were simply of the form `<R> <price> <C> <price> <P> <price>` where any of the clauses were optional we could read along, find either 'rental' or 'cermony' or 'catering' and then the next thing we extract would be the price for that aspect of the venue.  But if we got a string and the first thing we read was a price, we wouldn't know how it was applied until we kept reading **_holding the price in memory_** and discovered one of our keywords somewhere later in the string.  The difference here is, in the first example, **_state alone is enough_**, but in the second example **_you need to store something_**.  If you look back up at the Chomsky hierarchy, you'll notice that the most crude machine for regular grammars is a finite state machine, while the next level up is a push-down automaton.  It is push-down because it is employing a stack which is a simple way to implement hold-for-later in memory!  So in our example if you imagine each of the tokens in the string being popped off left to right, when we encounter a price without a keyword, we would need to push the price back onto the stack until we could pop both the price and the keyword so we know how the price is applied.

Here is some example output from a pushdown automaton that produces the averages of any indicated ranges for the three-tuple *(catering fee, ceremony fee, site rental fee)*

Pretty good for an oversimplified view.  In fact, it's greater than 99% good.  So when you are trying to extract information from text, which is the heart of language processing, give a lot of thought to exactly the information you are trying to extract and exactly what is needed to extract it, and no more.  Don't get too bogged down by the details.  As long as you see the snakes and trains and have the occasional juice box along the journey, you're doing pretty well.

### Harder, Better, Faster, Stronger

The above is a simple example of an *Information Extraction* (IE) system.  As we deal with more sophisticated examples of natural language, it will be helpful to think of text processing as a pipeline with roughly the following stages:

* Sentence cleaning and segmentation
* Sentence tokeniziation
* Part of speech tagging
* Entity detection
* Relationship detection

We've already been introduced to the first two in the last example.  Let's take a look at part of speech (POS) tagging.  The goal in POS tagging is to identify the parts of speech of a sentence.  Parts of speech are things like nouns, adjectives, personal pronouns, etc.  It is not always possible to identify the part of speech based on word lookup.  For example, in "The wind blew." and "Please wind the clock." lookup of wind would fail in one case unless the semantics of the sentence were analyzed.  Clearly, something more sophisticated than a dictionary lookup is needed in order to accurately identify which parts of speech a word is playing at the moment.  Part of speech taggers generally fall into one of two categories:

* Rule-based taggers
* Stochastic taggers

Rule based taggers attempt to build a set of rules that cover most cases.  Rules would be things like "*wind* means air movement if one of the adjacent word's root is *blow*".  Most rule-based taggers have hundreds of rules and use supervised learning techniques to minimize the error of the applied tags.

Stochastic taggers operate by building a model, usually a hidden markov model (HMM), that gives the most likely part of speech for a given word based on the path you traversed when encountering a word.  HMM's can be thought of as a state-transition diagram with probabilities along the transition edges.  So in our above example, it's likely the model would encounter the word *blew* and update it's part of speech prediction for the word *wind*.

You can see in both cases that some amount of training data is required in order to learn the appropriate rules or probabilities.  Both types of POS taggers tend to use supervised learning techniques or some form of expectation maximization, which is the typical algorithm in which most machine learning problems are formulated.

Alright, let's do some stuff with part of speech tagging...

In [82]:
from textblob import TextBlob
sample = 'A big yellow dog jumped over the moon.'
tb = TextBlob(sample)
print "Tags are:\n{}".format('\n'.join(str(i) for i in tb.tags))

Tags are:
(u'A', u'DT')
(u'big', u'JJ')
(u'yellow', u'JJ')
(u'dog', u'NN')
(u'jumped', u'VBD')
(u'over', u'IN')
(u'the', u'DT')
(u'moon', u'NN')


Our output is each token followed by the detected part of speech code in Penn Treebank format, courtesy of the University of Pennsyvania.  You can reference the complete list at 

https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

Let's extract the nouns and verbs, which by lookup are:

<pre>12.	NN	Noun, singular or mass
13.	NNS	Noun, plural
14.	NNP	Proper noun, singular
15.	NNPS	Proper noun, plural</pre>

Since they all begin with 'N', let's just check the first letter for compactness:

In [69]:
print( 'Just the nouns:  {}'.format( [ i[0] for i in tb.tags if i[1][0] == 'N' ] ) )

Just the nouns:  [u'dog', u'moon']


Similarly, verbs are codes that begin with 'V':

In [70]:
print( 'Just the verbs:  {}'.format( [ i[0] for i in tb.tags if i[1][0] == 'V' ] ) )

Just the verbs:  [u'jumped']


TextBlob is a package that provides a thin layer over Python's Natural Language Toolkit (NLTK).  It provides a lot of convenience functions as well as a number of taggers.  Here's a little test drive:

In [85]:
s = 'Strings of strings string together strings.'
print( '1: {}\n'.format(TextBlob(s).word_counts) )

s = 'First, I bought a car.  Then, I bought an air freshener.'
print( '2: {}\n'.format(TextBlob(s).sentences) )

s = '(512)555-1212'
print( '3: {}\n'.format(TextBlob(s).stripped) )

s = 'Joe and Mary had coffee.'
print( '4: {}\n'.format(', '.join(str(i) for i in TextBlob(s).ngrams())) )

1: defaultdict(<type 'int'>, {u'of': 1, u'strings': 3, u'together': 1, u'string': 1})

2: [Sentence("First, I bought a car."), Sentence("Then, I bought an air freshener.")]

3: 5125551212

4: ['Joe', 'and', 'Mary'], ['and', 'Mary', 'had'], ['Mary', 'had', 'coffee']



TextBlob even contains a built-in sentiment analyzer trained on a somewhat generic corpus.  If you were at the last meeting, you saw where I built a sentiment analyzer for movie ratings from scratch using a naive bayes classifier on a bag-of-words model.  While that is usually preferable, as you want results tailored to the corpus you are interacting with, it's nice to have simple built-ins to give you a chance to try some things out.

In [105]:
print( TextBlob( "I absolutely love this ski resort!" ).sentiment )
print( TextBlob( "I love most ski resorts." ).sentiment )
print( TextBlob( "Resort life is fine, I guess." ).sentiment )
print( TextBlob( "Skiing relies on gravity." ).sentiment )
print( TextBlob( "I'm really not that into skiing." ).sentiment )
print( TextBlob( "I hate skiing!" ).sentiment )

Sentiment(polarity=0.625, subjectivity=0.6)
Sentiment(polarity=0.5, subjectivity=0.55)
Sentiment(polarity=0.4166666666666667, subjectivity=0.5)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.1, subjectivity=0.2)
Sentiment(polarity=-1.0, subjectivity=0.9)


Polarity ranges from -1.0 being a very negative sentiment to 1.0 being a very positive sentiment.  Subjectivity is a measure of how objective/subjective the statement is from 0 (most objective) to 1.0 (most subjective).

### Noun Phrases

Once we have found the various parts of speech of our text, we can start aggregating them into entities that we can use in our IE machine.  Noun phrases are nouns paired with any modifiers like "yellow dog", "big purse", "urban outfit".  They tend to be things we are interested in products, movies, wedding venues, or anything else that is described with natural language.  Once we have a POS-tagged sentence, we can aggregate tags using a *chunker* into informational chunks that let us do more interesting things...  

In [73]:
sample = 'Ok Siri, whats that episode of Modern Family with Edward Norton?'
TextBlob.np_extractor.extract( sample )

[u'Ok Siri', u'Modern Family', u'Edward Norton']

...**_Detected OK Siri, search titles with tags [Modern Family, Edward Norton]_**...
<img src='mfedward.jpg'>

...*hold for applause*...

In [106]:
sample = 'I really want to have a wedding at an historic plantation house \
          with a waterfront backdrop.'
TextBlob.np_extractor.extract( sample )

[u'historic plantation house', u'waterfront backdrop']

Coupling this info with our sentiment analysis:

In [75]:
TextBlob(sample).sentiment

Sentiment(polarity=0.1, subjectivity=0.1)

We can start to build a picture of who people are based on the language they use, what things they are interested, and how they feel about those things.  Since noun phrases contain interesting nuggets of information in a natural language description, we can start to measure differences and similarities between natural language descriptions.  It stands to reason that the more interesting things two product descriptions have in common, especially if they are rarely ocurring things, the more similar the descriptions will be.  Here are some examples of high similarity discovered from our description data in our Concierge program using an algorithm I built that clusters by description similarity relying heavily on noun-phrase co-ocurrence:

[ "Where romance, elegance and history meet to create an exquisite facility for the wedding celebration of your dreams. Red Bank's landmark waterfront Molly Pitcher Inn prides itself in its impeccable hospitality services and is the recipient of the Five Star Diamond Award.","I love the Molly Pitcher Inn for your special day! This historical landmark located in Red Bank, NJ is absolutely lovely! Host your cocktail reception on their tented promenade overlooking the Navesink River and dance your wedding night away in the water view Ballroom for 200 guests. A great perk of this venue is the flexibility and customization to include, ceremony, reception, rehearsal dinner, bridal lunches and farewell breakfast, if you wish.",

"The boutique Oyster Point Hotel offers contemporary elegance, outstanding continental fusion cuisine and a staff dedicated to exceeding expectations to create the wedding of your dreams. The unparalleled sweeping views of the picturesque Navesink River will make your wedding a night to remember.","This venue is owned by the same as Molly Pitcher, it is a little more modern and sleek, very cool ballroom as well! It is also on the water in Red Bank and they have overnight accommodations for guests which is awesome. Their Riviera package has options for around $129 per person,  plus so many amazing options to work with your vision! I know Bill here, I can get you in contact with him anytime!" ]

--


[ "This plantation house is the epitome of southern elegance. The grounds are huge and the perfect place to set up a big white tent! Plus there are beautiful live oak trees to hang lanterns or twinkly lights for a romantic, rustic atmosphere.", 

"This charming waterfront venue is the perfect location for a breezy, southern reception. Located on gorgeous St. Helena Island, this plantation house is shaded by oak trees and sits on acres of marsh-front property. It also serves as an inn so you and your bridal party can stay for the weekend and have easy access to all the festivities." ]
 

In the next meeting, we'll talk more about entity recognition, discuss various techniques for building corpora and talk some about building the predictive models that underlie many of these algorithms.